In [34]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import math

%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 15, 6

from numpy.random import randn
np.random.seed(123)
import os
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))
np.set_printoptions(precision=4)
pd.options.display.max_rows = 10
#pd.options.display.max_rows = 20

#Pull in the file containing bike data
bike = ['id', 'start_date', 'start_station', 'end_date', 'end_station', 'duration_sec', 'blank1', 'is_member', 'blank2', 'blank3']
bike_data = pd.read_table('OD_2017 - Condensed.csv', sep=None, header=None, names=bike, skiprows=1, engine='python')
to_drop = ['blank1', 'blank2', 'blank3']
bike_data.drop(to_drop, inplace=True, axis=1)
bike_data.set_index(['start_date'])
bike_data['start_date'] = pd.to_datetime(bike_data['start_date'], yearfirst=True)
bike_data.set_index(['start_date'])

,id,start_station,end_date,end_station,duration_sec,is_member
start_date,,,,,,
2017-04-15 00:00:00,0.0,7060.0,2017-04-15 0:31,7060.0,1841.0,1.0
2017-04-15 00:01:00,1.0,6173.0,2017-04-15 0:10,6173.0,553.0,1.0
2017-04-15 00:01:00,2.0,6203.0,2017-04-15 0:04,6204.0,195.0,1.0
2017-04-15 00:01:00,3.0,6104.0,2017-04-15 0:06,6114.0,285.0,1.0
2017-04-15 00:01:00,4.0,6174.0,2017-04-15 0:11,6174.0,569.0,1.0
...,...,...,...,...,...,...
NaT,NaN,NaN,NaN,NaN,NaN,NaN
NaT,NaN,NaN,NaN,NaN,NaN,NaN
NaT,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
#Pull in station location data
location = ['code', 'latitude', 'longitude']
location_data = pd.read_table('Stations_2017.csv', sep=None, header=None, names=location, skiprows=1, engine='python')
location_data['code']=location_data['code'].astype(int)
location_data.set_index(['code'])

#isolate trips with different start and end locations and add lattitude and longitude info
trip_data=bike_data.loc[bike_data['start_station'] != bike_data['end_station']]
trip_data.dropna(inplace=True)
trip_data=location_data.merge(trip_data,how='left',left_on='code', right_on='start_station')
trip_data.rename(columns={'latitude': 'start_latitude', 'longitude': 'start_longitude'}, inplace=True)
trip_data=location_data.merge(trip_data,how='left',left_on='code', right_on='end_station')
trip_data.rename(columns={'latitude': 'end_latitude', 'longitude': 'end_longitude'}, inplace=True)
trip_data.dropna(inplace=True)
trip_data = trip_data.loc[:, ['id','start_latitude','start_longitude','end_latitude','end_longitude']]
trip_data

#calculate distances travelled for each trip with a different start and end station

distance_array = []

def calc_distance(lat1, lon1, lat2, lon2):
    earthRadiusKm = 6371
    dLat = math.radians(lat2-lat1)
    dLon = math.radians(lon2-lon1)
    lat1 = math.radians(lat1)
    lat2 = math.radians(lat2)
    a = math.sin(dLat/2) * math.sin(dLat/2) + math.sin(dLon/2) * math.sin(dLon/2) * math.cos(lat1) * math.cos(lat2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    return round(earthRadiusKm * c,4)

def get_distances():
    
    last=len(trip_data.index)
    
    for i in range(last):
        trip_id=trip_data.loc[i,'id']
        lat1=trip_data.loc[i,'start_latitude']
        lat2=trip_data.loc[i,'end_latitude']
        lon1=trip_data.loc[i,'start_longitude']
        lon2=trip_data.loc[i,'end_longitude']
        distance=calc_distance(lat1,lon1,lat2,lon2)
        distance_array.append(distance)
            
get_distances()

#convert the distance array to a usable format
distance_data = pd.DataFrame(data=distance_array)
distance_data

#put the pieces back together into a final data frame
trip_data_final = pd.DataFrame({'distance':distance_data.loc[:,0],'end_date':bike_data.end_date,'start_station':bike_data.start_station,'end_station':bike_data.end_station, 'duration': bike_data.duration_sec, 'is_member':bike_data.is_member, 'id':bike_data.id, 'start_date':bike_data.start_date})
trip_data_final.dropna(inplace=True)
trip_data_final
trip_data_final = trip_data_final.set_index(['start_date'])

C:\Users\Choy\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [36]:
trip_data_final

,distance,duration,end_date,end_station,id,is_member,start_station
start_date,,,,,,,
2017-04-15 00:00:00,1.2700,1841.0,2017-04-15 0:31,7060.0,0.0,1.0,7060.0
2017-04-15 00:01:00,1.2700,553.0,2017-04-15 0:10,6173.0,1.0,1.0,6173.0
2017-04-15 00:01:00,1.2700,195.0,2017-04-15 0:04,6204.0,2.0,1.0,6203.0
2017-04-15 00:01:00,1.2201,285.0,2017-04-15 0:06,6114.0,3.0,1.0,6104.0
2017-04-15 00:01:00,1.2201,569.0,2017-04-15 0:11,6174.0,4.0,1.0,6174.0
...,...,...,...,...,...,...,...
2017-05-04 15:19:00,4.6388,1768.0,2017-05-04 15:49,6314.0,244115.0,1.0,6092.0
2017-05-04 15:19:00,2.9152,958.0,2017-05-04 15:35,6434.0,244116.0,1.0,6042.0
2017-05-04 15:19:00,2.9152,412.0,2017-05-04 15:26,6132.0,244117.0,1.0,6134.0


In [37]:
#Pull in file containing temperature data and resample
temperature = ['datetime', 'Montreal']
temperature_data = pd.read_table('Temperature - Montreal.csv', sep=None, header=None, names=temperature, skiprows=1, engine='python')
temperature_data['datetime'] = pd.to_datetime(temperature_data['datetime'], yearfirst=True)
temperature_data = temperature_data.set_index(['datetime'])
temperature_data = temperature_data.resample('1min').pad()
temperature_data = temperature_data.rename(index=str, columns={'Montreal':'Temperature'})

In [38]:
#Pull in file containing humidity data and resample
humidity = ['datetime', 'Montreal']
humidity_data = pd.read_table('Humidity - Montreal.csv', sep=None, header=None, names=humidity, skiprows=1, engine='python')
humidity_data['datetime'] = pd.to_datetime(humidity_data['datetime'], yearfirst=True)
humidity_data = humidity_data.set_index(['datetime'])
humidity_data = humidity_data.resample('1min').pad()
humidity_data = humidity_data.rename(index=str, columns={'Montreal':'Humidity'})

In [39]:
master = pd.merge(trip_data_final, temperature_data, how = 'inner', left_index=True, right_index=True)

In [40]:
master = pd.merge(master, humidity_data, how = 'inner', left_index=True, right_index=True)

In [41]:
#check for any NaNs
master.isnull().sum(), len(master)

(distance         0
 duration         0
 end_date         0
 end_station      0
 id               0
 is_member        0
 start_station    0
 Temperature      0
 Humidity         0
 dtype: int64, 244120)

In [42]:
master

,distance,duration,end_date,end_station,id,is_member,start_station,Temperature,Humidity
2017-04-15 00:00:00,1.2700,1841.0,2017-04-15 0:31,7060.0,0.0,1.0,7060.0,13.0,22
2017-04-15 00:01:00,1.2700,553.0,2017-04-15 0:10,6173.0,1.0,1.0,6173.0,13.0,22
2017-04-15 00:01:00,1.2700,195.0,2017-04-15 0:04,6204.0,2.0,1.0,6203.0,13.0,22
2017-04-15 00:01:00,1.2201,285.0,2017-04-15 0:06,6114.0,3.0,1.0,6104.0,13.0,22
2017-04-15 00:01:00,1.2201,569.0,2017-04-15 0:11,6174.0,4.0,1.0,6174.0,13.0,22
...,...,...,...,...,...,...,...,...,...
2017-05-04 15:19:00,4.6388,1768.0,2017-05-04 15:49,6314.0,244115.0,1.0,6092.0,10.0,82
2017-05-04 15:19:00,2.9152,958.0,2017-05-04 15:35,6434.0,244116.0,1.0,6042.0,10.0,82
2017-05-04 15:19:00,2.9152,412.0,2017-05-04 15:26,6132.0,244117.0,1.0,6134.0,10.0,82
2017-05-04 15:19:00,2.9152,208.0,2017-05-04 15:23,6235.0,244118.0,1.0,6227.0,10.0,82


In [43]:
from pandas import DataFrame

In [49]:
type(master)

pandas.core.frame.DataFrame

In [50]:
Master_df = DataFrame(master)

In [51]:
master.to_excel('test.xlsx',sheet_name='sheet1',index=False)

PermissionError: [Errno 13] Permission denied: 'test.xlsx'